In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from scipy.stats import boxcox
from scipy.special import inv_boxcox

from sklearn.linear_model import RidgeCV, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [51]:
df = pd.read_csv('data/clean_data.csv')
df.head()

,SalePrice,Overall Qual,Gr Liv Area,Garage Area,Total Bsmt SF,Year Built,Full Bath,Fireplaces,Mas Vnr Area,Lot Frontage,...,Garage Type_NA,Paved Drive_N,Paved Drive_P,Paved Drive_Y,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,215000,6,1656,528.0,1080.0,1960,1,2,112.0,141.0,...,0,0,1,0,0,0,0,0,1,0
1,105000,5,896,730.0,882.0,1961,1,0,0.0,80.0,...,0,0,0,1,0,0,0,0,1,0
2,172000,6,1329,312.0,1329.0,1958,1,0,108.0,81.0,...,0,0,0,1,0,0,0,0,1,0
3,244000,7,2110,522.0,2110.0,1968,2,2,0.0,93.0,...,0,0,0,1,0,0,0,0,1,0
4,189900,5,1629,482.0,928.0,1997,2,1,0.0,74.0,...,0,0,0,1,0,0,0,0,1,0


In [52]:
y = df['SalePrice']
X = df.drop(columns=['SalePrice'])

Выделим числовые признаки.

In [53]:
num_features = []

for feature in list(X):
    if X[feature].nunique() > 10:
        num_features.append(feature)
        
print(*num_features, sep=', ')

Gr Liv Area, Garage Area, Total Bsmt SF, Year Built, Mas Vnr Area, Lot Frontage, Open Porch SF


## Построение baseline модели

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

ss = StandardScaler()
X_train[num_features] = ss.fit_transform(X_train[num_features])
X_test[num_features] = ss.fit_transform(X_test[num_features])

In [43]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     136.0
Date:                Sun, 21 Mar 2021   Prob (F-statistic):               0.00
Time:                        21:56:15   Log-Likelihood:                -23031.
No. Observations:                2020   AIC:                         4.635e+04
Df Residuals:                    1876   BIC:                         4.716e+04
Df Model:                         143                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   5125

In [44]:
X_test = sm.add_constant(X_test)
preds = results.predict(X_test)

print('Test MSE = {}'.format(round(sum((preds - y_test)**2) / len(preds)), 3))

Test MSE = 506944223


Результаты показывают, что $R^2$ достигает значения *0.912*.

Можно увидеть, что веса перед признаками и их стандартные отклонения чересчур большие. Кроме того, модель явно говорит нам о том, что имеет место сильная мультиколлинеарность.

Также мало интерпретируемым является значение *MSE*, посчитанное на ценах.

В связи с этим возникают следующие идеи:
* привести распределение целевой переменной к нормальному с помощью преобразования Бокса-Кокса (это так же повысит интепретируемость *MSE* из-за уменьшения разброса значений *SalePrice*);
* проблему мультиколлинеарности может помочь решить *l2*-регуляризация;
* для поиска оптимального значения коэффициента регуляризации можно воспользоваться кросс-валидацией.

## Улучшение бейзлайна

Применим преобразование Бокса-Кокса.

In [54]:
y, lmbda = boxcox(y, lmbda=None)
print('Оптимальное лямбда = {}'.format(round(lmbda, 2)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

ss = StandardScaler()
X_train[num_features] = ss.fit_transform(X_train[num_features])
X_test[num_features] = ss.fit_transform(X_test[num_features])

Оптимальное лямбда = 0.09


In [55]:
alphas = np.logspace(-2, 2, num=10)
model = RidgeCV(alphas=alphas, cv=10).fit(X_train, y_train)

In [56]:
preds = model.predict(X_test)

print('Оптимальное alpha = {}'.format(round(model.alpha_, 1)))
print('R**2 на тесте = {}'.format(round(model.score(X_test, y_test), 3)))
print('Test MSE = {}.'.format(round(sum((preds - y_test)**2 / len(preds), 3)))
print('Максимальный и минимальный веса модели: {} и {}'.format(round(max(model.coef_), 2), 
                                                               round(min(model.coef_), 2)))

Оптимальное alpha = 35.9
R**2 на тесте = 0.918
Test MSE = 0.215.
Максимальный и минимальный веса модели: 0.35 и -0.21


Можно немного уточнить параметр $\alpha$.

In [57]:
alphas = np.linspace(10, 50, num=20)
model = RidgeCV(alphas=alphas, cv=10).fit(X_train, y_train)

In [58]:
preds = model.predict(X_test)

print('Оптимальное alpha = {}'.format(round(model.alpha_, 1)))
print('R**2 на тесте = {}'.format(round(model.score(X_test, y_test), 3)))
print('Test MSE = {}.'.format(round(sum((preds - y_test)**2) / len(preds), 3)))
print('Максимальный и минимальный веса модели: {} и {}'.format(round(max(model.coef_), 2), 
                                                               round(min(model.coef_), 2)))

Оптимальное alpha = 31.1
R**2 на тесте = 0.918
Test MSE = 0.217.
Максимальный и минимальный веса модели: 0.35 и -0.22


Видно, что описанные выше идеи сработали, и теперь можно обучать финальную модель.

In [62]:
model = Ridge().fit(X_train, y_train)

In [68]:
preds = model.predict(X_test)

print('R**2 на тесте = {}'.format(round(model.score(X_test, y_test), 3)))
print('Test MSE = {}.'.format(round(sum((preds - y_test)**2) / len(preds), 3)))
print('Максимальный и минимальный веса модели: {} и {}'.format(round(max(model.coef_), 2), 
                                                               round(min(model.coef_), 2)))

R**2 на тесте = 0.918
Test MSE = 0.103.
Максимальный и минимальный веса модели: 0.96 и -0.86


In [67]:
a = inv_boxcox(preds, lmbda)
b = inv_boxcox(y_test, lmbda)

a = np.log(a)
b = np.log(b)

round(sum((a - b)**2) / len(a), 3)

0.013

In [66]:
model.coef_

array([ 0.00000000e+00,  1.73500469e-01,  3.60026265e-01,  7.01385505e-02,
        7.69862925e-02,  1.04102652e-01,  8.57155725e-03,  9.01452726e-02,
        1.17612252e-02,  2.72070741e-02,  1.41574192e-02,  3.22746279e-02,
        1.25505711e-01,  2.71491526e-02,  2.56829305e-02,  6.26972018e-02,
        4.30153179e-02,  2.99024084e-03,  5.90572431e-02,  1.22028225e-01,
        2.10398054e-02, -4.92719085e-02,  0.00000000e+00, -2.51136078e-01,
       -2.16227580e-01,  5.92292448e-02,  1.39121560e-01, -7.48219452e-02,
        8.62790960e-02,  1.88351117e-02,  8.47466927e-02,  4.91403091e-02,
        1.41218143e-01,  1.21792902e-01, -9.70202873e-02,  3.99846883e-02,
       -5.18699490e-02, -8.56987645e-01,  4.26725018e-02,  3.07717587e-01,
       -2.79164780e-01,  2.94593636e-01,  2.97539710e-01,  1.93628989e-01,
       -3.16306840e-02,  1.03495829e-01, -3.33477716e-02, -3.85173731e-02,
       -6.15581762e-02,  3.67848163e-02,  1.31948507e-02,  1.15785092e-02,
        2.25231538e-02,  